# BPO

In [1]:
from koselleck import *

In [2]:
# get_numlines??

In [3]:

def get_skipgrams(idir=PATH_SKIPGRAMS_YR,skipgram_n=25):
    odf=pd.DataFrame([
        {
            'corpus':fn.split('.')[2],
            'year':int([x for x in fn.split('.') if x.isdigit()][0]),
#             'period_end':int([x for x in fn.split('.') if x.isdigit()][-1]),
            'path':os.path.join(idir,fn)
        }
        for fn in os.listdir(idir)
        if fn.startswith('data.skipgrams')
    ]).sort_values(['corpus','year'])

#     odf['num_lines']=odf.path.progress_apply(get_numlines)
#     odf['num_words']=odf['num_lines']*skipgram_n
    return odf#.query('1700<=year')

In [4]:
dfskip=get_skipgrams()
dfskip

,corpus,year,path
182,bpo,1681,/home/ryan/github/koselleck/data/skipgrams/yea...
56,bpo,1682,/home/ryan/github/koselleck/data/skipgrams/yea...
51,bpo,1683,/home/ryan/github/koselleck/data/skipgrams/yea...
60,bpo,1684,/home/ryan/github/koselleck/data/skipgrams/yea...
200,bpo,1685,/home/ryan/github/koselleck/data/skipgrams/yea...
...,...,...,...
116,bpo,1967,/home/ryan/github/koselleck/data/skipgrams/yea...
20,bpo,1968,/home/ryan/github/koselleck/data/skipgrams/yea...
34,bpo,1969,/home/ryan/github/koselleck/data/skipgrams/yea...
87,bpo,1970,/home/ryan/github/koselleck/data/skipgrams/yea...


In [5]:
def periodize(y,ybin=10):
    y1=y//ybin*ybin
    y2=y1+ybin
    return f'{y1}-{y2}'

In [6]:
periodize(1705)

'1700-1710'

In [7]:
dfskip['period']=dfskip.year.apply(periodize_sattelzeit_binary)
dfskip[~dfskip.period.isnull()]

,corpus,year,path,period
157,bpo,1700,/home/ryan/github/koselleck/data/skipgrams/yea...,1700-1770
42,bpo,1701,/home/ryan/github/koselleck/data/skipgrams/yea...,1700-1770
25,bpo,1702,/home/ryan/github/koselleck/data/skipgrams/yea...,1700-1770
288,bpo,1703,/home/ryan/github/koselleck/data/skipgrams/yea...,1700-1770
69,bpo,1704,/home/ryan/github/koselleck/data/skipgrams/yea...,1700-1770
...,...,...,...,...
21,bpo,1895,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900
108,bpo,1896,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900
17,bpo,1897,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900
285,bpo,1898,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900


In [8]:
dfskipruns=pd.concat([
    dfskip.assign(run=f'run_{str(i+1).zfill(2)}')
    for i in range(25)
])
dfskipruns

,corpus,year,path,period,run
182,bpo,1681,/home/ryan/github/koselleck/data/skipgrams/yea...,None,run_01
56,bpo,1682,/home/ryan/github/koselleck/data/skipgrams/yea...,None,run_01
51,bpo,1683,/home/ryan/github/koselleck/data/skipgrams/yea...,None,run_01
60,bpo,1684,/home/ryan/github/koselleck/data/skipgrams/yea...,None,run_01
200,bpo,1685,/home/ryan/github/koselleck/data/skipgrams/yea...,None,run_01
...,...,...,...,...,...
116,bpo,1967,/home/ryan/github/koselleck/data/skipgrams/yea...,None,run_25
20,bpo,1968,/home/ryan/github/koselleck/data/skipgrams/yea...,None,run_25
34,bpo,1969,/home/ryan/github/koselleck/data/skipgrams/yea...,None,run_25
87,bpo,1970,/home/ryan/github/koselleck/data/skipgrams/yea...,None,run_25


In [9]:
# disable_gensim_logging()
enable_gensim_logging()

In [10]:
from gensim.models import FastText
from gensim.models import Word2Vec


def gen_and_save_model(dfskip, nskip=25000, force=False):
    row=dfskip.iloc[0]
    odir=os.path.join(PATH_MODELS_NEW,row.corpus,row.period,row.run)
    ofnfn=os.path.join(odir,'model.bin')
    if force or not os.path.exists(ofnfn):
        ensure_dir_exists(odir)
        ss=SkipgramsSamplers(dfskip.path, nskip)
        model = Word2Vec(vector_size=300, window=10, min_count=50, sentences=ss, epochs=5, workers=8)
        model.save(ofnfn)
    return pd.DataFrame()

# model.wv.most_similar('value',topn=25)

In [11]:
dfskipdo=dfskipruns.query('period=="1830-1900" & run>"run_10"')
dfskipdo

,corpus,year,path,period,run
198,bpo,1830,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900,run_11
220,bpo,1831,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900,run_11
163,bpo,1832,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900,run_11
269,bpo,1833,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900,run_11
11,bpo,1834,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900,run_11
...,...,...,...,...,...
21,bpo,1895,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900,run_25
108,bpo,1896,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900,run_25
17,bpo,1897,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900,run_25
285,bpo,1898,/home/ryan/github/koselleck/data/skipgrams/yea...,1830-1900,run_25


In [13]:
pmap_groups(gen_and_save_model, dfskipdo.groupby(['period','run']), num_proc=1, kwargs=dict(force=True))

Mapping gen_and_save_model [x1]:   0%|          | 0/15 [00:00<?, ?it/s]collecting all words and their counts
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #10000, processed 429632 words, keeping 25581 word types
PROGRESS: at sentence #20000, processed 848267 words, keeping 34425 word types
PROGRESS: at sentence #30000, processed 1260465 words, keeping 40480 word types
PROGRESS: at sentence #40000, processed 1642149 words, keeping 45131 word types
PROGRESS: at sentence #50000, processed 2063202 words, keeping 48959 word types
PROGRESS: at sentence #60000, processed 2459522 words, keeping 53331 word types
PROGRESS: at sentence #70000, processed 2908360 words, keeping 56871 word types
PROGRESS: at sentence #80000, processed 3339710 words, keeping 60549 word types
PROGRESS: at sentence #90000, processed 3759609 words, keeping 64537 word types
PROGRESS: at sentence #100000, processed 4182982 words, keeping 67548 word types
PROGRESS: at sentence #110

KeyboardInterrupt: 

In [ ]:
from gensim.models import KeyedVectors
m=KeyedVectors.load('/home/ryan/github/koselleck/data/models/bpo/1700-1770/run_01/model.bin')
m

In [ ]:
m.wv.most_similar('history')